This notebook takes borehole data from the East Kimberley, interpolates/ averages the data so that it they can be formulated as a sample vs feature array. This array can be used to investigate correlations between datasets, impute the array  and be used as training data for interpretations at boreholes

Neil Symington
neil.symington@ga.gov.au

In [5]:
from sqlite3 import dbapi2 as sqlite
import pandas as pd
import numpy as np
from hydrogeol_utils import spatial_functions
import os
import sqlalchemy as db

In [6]:
# Extract data from the database

path = r"\\prod.lan\active\proj\futurex\East_Kimberley\Working\SharedWorkspace\Bores_working\compilation\spatialite"

DB_PATH = os.path.join("East_Kimberley_borehole_data.sqlite")

engine = db.create_engine('sqlite:///' + DB_PATH, module=sqlite)

connection = engine.connect()


In [7]:
# Import the header table

query = 